<a href="https://colab.research.google.com/github/vandalt/phy3051-students-private/blob/main/tp11-transformers/huggingface_blank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Huggingface

Dans le notebook précédent, nous avons vu comment un transformer peut-être créé et modifié directement avec PyTorch.
On aurait également pu aller un pas plus loin et implémenter le modèle nous même avec PyTorch.
C'est un exercice intéressant, mais qui demande un peu plus de temps que ce que nous avons dans les TPs.
Ceci dit, je vous encourage à consulter des ressources en ligne à ce sujet si ça vous intéresse (voir le notebook PyTorch).

Ici, nous explorerons plutôt [_huggingface_](https://huggingface.co/).
Huggingface est un peu comme un GitHub pour les modèles d'IA.
Les utilisateurs peuvent publier l'architecture de leur modèle, des poids pré-entraînés et des ensembles de données.
Le site fourni aussi des librairies implémentant certains communs, notamment des [transformers](https://huggingface.co/docs/transformers/index) et des [modèles de diffusion](https://huggingface.co/docs/diffusers/index).
Voir la section [documentation](https://huggingface.co/docs) pour plus de détails.
Dans ce notebook, nous allons nous familiariser avec la librairie [transformers](https://huggingface.co/docs/transformers/index).

La librairie `transformers` va un peu dans la direction opposée d'une implémentation complète en PyTorch: presque toutes les opérations sont cachées derrières des classes nous permettant simplement de spécifier les paramètres de notre modèle.
Ce n'est pas la meilleure façon de comprendre tous les détails d'un modèle, mais c'est pratique pour le tester rapidement et comprendre comment il interprète les données.
De plus, tous les modèles de la librairie sont disponible [sur GitHub](https://github.com/huggingface/transformers/tree/main/src/transformers/models).
N'hésitez pas à les consulter!

## Installation

Pour utiliser `transformers`, il faudra d'abord l'installer. Nous installerons du même coup les autres librries huggingface utilisées dans ce TP.

In [ ]:
INSTALL = False
if INSTALL:
    !python -m pip -q install transformers datasets evaluate
else:
    print("Skip install")

Si vous n'avez pas de GPU, vous pouvez remplacer `transformers` par `transformers[torch]`.

## Connexion

L'accès à certains modèles Huggingface requiert un compte et une authentification.
Pour se connecter, on peut utiliser `notebook_login()` dans un notebook et `login()` dans un terminal

In [ ]:
def is_notebook() -> bool:
    # https://stackoverflow.com/questions/15411967/how-can-i-check-if-code-is-executed-in-the-ipython-notebook
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

NOTEBOOK = is_notebook()

In [ ]:
from huggingface_hub import notebook_login, login

if NOTEBOOK:
    notebook_login()
else:
    login()

## Pipelines

L'interface la plus simple de `transformers` est la classe `Pipeline`.
Celle-ci nous permet d'importer et d'utiliser un transformeur en trois lignes de code!

### Génération de texte

Bien que ce ne soit pas de l'analyse de données physique, la génération de texte est tellement omni-présente dans les dernières années qu'il peut être intéressant de voir comment l'appliquer avec Huggingface.

Le modèle [Gemma](https://huggingface.co/google/gemma-3-1b-it) de Google requiert une authentification avec les cellules ci-dessus. Si vous ne souhaitez pas vous authentifier, décommentez la deuxième ligne. C'est GPT-2 qui est utilisé par défaut.

J'ai également inclut deux exemples. L'un qui simule un chat-bot et l'autre qui demande simplement de compléter une phrase. Les deux fonctionnent avec Gemma, mais seulement le 2e avec GPT-2.

In [ ]:
from transformers import pipeline

txt_pipeline = pipeline(task="text-generation", model="google/gemma-3-1b-it")
# txt_pipeline = pipeline(task="text-generation")

In [ ]:
messages = [
    [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a helpful assistant."},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": "What is Markov Chain Monte Carlo? Explain in two sentences."},]
        },
    ],
]
# messages = "Markov Chain Monte Carlo is an inference method that"

In [ ]:
simple = isinstance(messages, str)
reply = txt_pipeline(messages, max_new_tokens=100)
if simple:
    print(reply[0]["generated_text"])
else:
    print(reply[0][0]["generated_text"][2]["content"])

Ces modèles sont un peu volumineux. On peut simplement le supprimer.

In [ ]:
del txt_pipeline

### Classification d'image

L'interface `pipeline` ne se limite bien sûr pas à la génération de texte.
On peut spécifier une autre tâche via le premier argument, `task`.
Par exemple, pour classifier des images on utiliserait `task="image-classification"`.
Le modèle par défaut est le transformeur visuel `vit` avec des sous-images de 16 et une taille initiale de 224x224 pixels, soit ([google/vit-base-patch16-224](https://huggingface.co/google/vit-base-patch16-224)).
On spécifie l'argument `model` ci-dessous pour clarifier le modèle utilisé.

**Exercice: Créez un pipeline destiné à la classification d'image et appliquez le à n'importe quelle image trouvée en ligne. Il suffit de passer le lien en argument au pipeline. N'oubliez pas de supprimer le pipeline ensuite.**

## Interface complète

Le `pipeline` ci-dessus nous permet de tester un modèle très rapidement, mais ne permet pas d'interagir avec un ensemble de données ou d'entraîner le modèle.

### Importation des données

Comme avec PyTorch, huggingface comprends plusieurs ensembles de données.
Pour y accéder, on peut utiliser la librairie [Datasets](https://huggingface.co/docs/datasets/index).

Ici, on importe seulement les 5000 premiers exemples des données d'entraînement pour réduire la taille des fichiers sur notre disque.
On pourra créer nos propre sous-ensembles à partir des données d'entraînement uniquement.

**Exercice: Explorez les datasets Huggingface et choisissez en un pour cet exemple. Vous pouvez utiliser `split="train[:5000]"` pour importer uniquement 5000 exemples.**

In [ ]:
from datasets import load_dataset

# TODO: Import dataset

On voit que les données contiennent des images et leurs annotation.
Séparons maintenant le tout avec 80% des exemples utilisés dans l'entraînement et le dernier 20% utilisé pour la validation.

**Exercice: Utilisez la méthode `train_test_split` pour garder 80% des données dans l'ensemble d'entraînement.**

In [ ]:
# TODO: Split

Voyons voir de quoi a l'air un exemple.

In [ ]:
data["train"][0]

On voit que:

- Contrairement à PyTorch, qui nous donne des tuples, on a ici un dictionnaire.
- L'image est au format PIL, que nous avons vu plus tôt dans le cours
- L'annotation est un nombre entier, comme avec PyTorch

L'attribut `features` des données nous permet cependant d'accéder à un peu plus de détail sur les données

**Exercice: Explorez l'attribut `features` des données d'entraînement et affichez:**

- Le nom de la classe 63
- Le nombre associé à la classe "steak"
- Le nombre total de classes

In [ ]:
test_num = 53
test_name = "steak"
# TODO: Labels

**Exercice: à partir des méthodes ci-dessus, créez un dictionnaire label2idx et un dictionnaire idx2label.**

In [ ]:
# TODO: Create classes

**Exercice: Affichez un exemple aléatoire tiré des données d'entraînement et affichez le nom de sa classe.**

In [ ]:
# TODO: Display

### Préparation des données

Comme avec PyTorch, il faut transformer les données de PIL vers des tenseurs.
Pour ce faire, Hugginface inclut des classes de type `Preprocessor`.
On peut utiliser le pre-processeur d'un modèle pré-entraîné, par exemple ViT entraîné sur les données ImageNet-21K.

In [ ]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)
image_processor

On peut ensuite utiliser la classe `Preprocessor` pour créer des transformations PyTorch.

**Exercice: Utilisez les attributs d'`image_processor` pour créer des transformations PyTorch qui permettront de convertir les données au format attendu par PyTorch. Utilisez Compose pour grouper les transformations suivantes:**

- Une modification de la taille à 224 pixels, ou optionnellement un découpage aléatoire.
- Une transformation en tenseur
- Une normalisation

La fonction `transforms` et la méthode `with_transform` permet d'appliquer les transformations aux données.

In [ ]:
from torchvision.transforms import ToTensor, Compose, Resize, Normalize, RandomResizedCrop

size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
# TODO: Ajouter transformations
torch_transforms = Compose([
])
def transforms(examples):
    examples["pixel_values"] = [torch_transforms(img) for img in examples["image"]]
    del examples["image"]
    return examples
data = data.with_transform(transforms)

**Exercice: Affichez un exemple des données pour voir comment elles ont été transformées**

On peut aussi définitir un objet [`DataCollator`](https://huggingface.co/docs/transformers/en/main_classes/data_collator). Ceux-ci permettent de convertir les données en sous-ensemble lors de l'etraînement, un peu comme un `DataLoader` dans PyTorch.

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

### Création du modèle

Plus haut, nous avons importé notre modèle via un `pipeline`.
Ici, nous allons plutôt importer le modèle directement.
Nous utiliserons tout de même un modèle pré-entraîné.

**Exercice: Utilisez la méthode `from_pretrained` et le `checkpoint` défini ci-dessus pour créer un modèle. Affichez le modèle ensuite.**

In [ ]:
from transformers import AutoModelForImageClassification

# TODO: Create model

L'avertissement ci-dessus nous indique que bien que le modèle soit pré-entraîné, son classificateur (la dernière couche) n'est pas entraîné.
Il faudra donc ajuster les poids et biais à la tâche qui nous intéresse ici.
Par contre, tout le reste du modèle est pré-entraîné.

Voyons voir de quoi est fait le modèle.

In [ ]:
# TODO: Display

Remarquez que la structure générale est la même que celle vue en classe et dans le notebook PyTorch:

- Un encodage des images et la position
- Un dropout optionnel
- Un encodeur, composé ici de 12 blocs ViT, qui eux contiennent:
  - Une couche d'attention
  - Une connection résiduelle
  - Une couche pleinement connectée
  - Des normalisations de couche (`LayerNorm`)
- Une classificateur permettant de convertir la sortie la sortie du classificateur en score pour chaque catégorie

Par défaut, le modèle contient uniquement deux sorties.
Il faut l'initialiser avec le bon nombre de classes.
On peut également utiliser la conversion entre les numéros de classes et leur nom.

**Exercice: Créez un nouveau modèle, mais cette fois avec le bon nombre de classes. Utilisez également les arguments `id2label` et `label2id`.**

In [ ]:
# TODO: Create for num_labels

In [ ]:
# TODO: Display

### Entraînement

Une fois le modèle définit, on peut l'entraîner.
Il y aura deux étapes:

1. Définir une métrique d'évaluation
2. Définir une boucle d'entraînement et l'exécuter.

#### Métrique d'évaluation

Dans Huggingface, c'est la librairie [`evaluate`](https://huggingface.co/docs/evaluate/index) qui définit les métriques permettant d'évaluer la qualité d'un modèle. Voir ce lien pour une liste des métriques: <https://huggingface.co/evaluate-metric>.

Ici, on utilise la métrique de précision (`accuracy`)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
accuracy

In [ ]:
def compute_metrics(eval_pred):
    """
    Fonction pour évaluer la précision à partir d'un ensemble de prédictions
    et de classes de référence.
    """
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    return accuracy.compute(predictions=preds, references=labels)

#### Boucle d'entraînement

On peut ensuite implémenter une boucle d'entraînement. L'interface est différente de PyTorch, mais on reconnaît la terminologie de plusieurs arguments.

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
training_args = TrainingArguments(
    output_dir="new_model",
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    report_to="none",
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    processing_class=image_processor,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("new_model")

### Inférence

Une fois le modèle entraîné, on peut l'importer via un pipeline et l'utiliser sur une image. On peut le faire via un pipeline ou en passant directement l'image au modèle. La 2e option est très similaire à l'exemple PyTorch du notebook précédent.

**Exercice: Importez des données de validation pour l'ensemble de données utilisé.**

In [ ]:
valid_data = load_dataset("food101", split="validation[:100]")

In [ ]:
valid_data

**Exercice: Affichez un exemple aléatoire**

In [ ]:
idx = int(rng.integers(valid_data.num_rows))
eg = valid_data[idx]
img, label = eg["image"], eg["label"]

In [ ]:
plt.imshow(img)
plt.title(idx2label[label])
plt.show()

**Exercice: Créez un pipeline avec votre modèle et testez avec un exemple de validation, puis avec un exemple trouvé en ligne.**

## Références

- Tutoriel Huggingface duquel celui-ci est inspiré: <https://huggingface.co/docs/transformers/tasks/image_classification>